In [9]:
from transformers import TFMarianMTModel, MarianTokenizer, Trainer, TrainingArguments
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

import pandas as pd
from datasets import load_dataset

In [12]:
model_name = 'Helsinki-NLP/opus-mt-en-fr'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = TFMarianMTModel.from_pretrained(model_name)

tf_model.h5:   0%|          | 0.00/301M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/Helsinki-NLP/opus-mt-en-fr/c860b5d50a25f2c6d063886cf36720016387b0ee94e7d33487bad4037b3e9ff6?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tf_model.h5%3B+filename%3D%22tf_model.h5%22%3B&Expires=1718165651&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxODE2NTY1MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9IZWxzaW5raS1OTFAvb3B1cy1tdC1lbi1mci9jODYwYjVkNTBhMjVmMmM2ZDA2Mzg4NmNmMzY3MjAwMTYzODdiMGVlOTRlN2QzMzQ4N2JhZDQwMzdiM2U5ZmY2P3Jlc3BvbnNlLWNvbnRlbnQtZGlzcG9zaXRpb249KiJ9XX0_&Signature=K9xZf%7E2PsDTUzmBDp40LMNm-eW33HSg-pQn9Ye1gKbygdrzOHqx3FOtJW%7EjK-2XGaOHBlimJKbLhFbUGN0MpAzGro9GHc6BiOQE0FfyBG38yLWKQ7ngn24GxyGx9qqzCSdwydUWsh4b9qA1-EvK-SalpjQDF-asDVnSDK-%7E7IheJLxE0i983mc2yBFWJvKyYHQjYzFALYkuRHV1YR2jTQnLg3V3uFMsIdt22ccJsjIQYAfb9R2Jc3idoZPQ72d5d3mArpehYcRMoKsj5PHnB6-2trjp4A%7EAkddM19dvU1i7geuH0Co9aS9AUBtGGLt8eMCzcyRnchqEkXJiFe8EcYQ__&Key-Pair-Id=KVTP0A1DKRTAX

ReadTimeout: (ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: c90afaba-5049-4cef-b44d-e374cf7be5ab)')

In [ ]:
def comma_split(row):
    if ',' in row['English']:
        new_row = {'English': row['English'].split(',')[0], 'Edo': row['Edo']}
        new_row2 = {'English': row['English'].split(',')[1], 'Edo': row['Edo']}
        words.drop(row, inplace = True)
        words.iloc[words.shape[0]] = new_row
        words.iloc[words.shape[0]] = new_row2
        

In [ ]:
data = pd.read_excel('edo_english.xlsx')
sentences = data[["Edo Sentences", "English Translations"]]
words = data[['English', 'Edo']]
words.columns = sentences.columns
dataset = pd.concat([sentences, words])
dataset

In [ ]:
def tokenize(data):
    inputs = tokenizer(data['input_text'], padding="max_length", truncation=True)
    targets = tokenizer(data['target_text'], padding="max_length", truncation=True)
    inputs['labels'] = targets['input_ids']
    return inputs

In [ ]:
dataset = dataset.map(tokenize)

In [ ]:
# dataset = load_dataset('csv', data_files={'train': 'gs://your-bucket-name/train.csv', 'validation': 'gs://your-bucket-name/validation.csv'})


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
)

In [ ]:
trainer.train()

In [ ]:
trainer.save('model')